## Background
We assume the system is $x \in R^n$ and $x[t+1] = Ax[t] + Bu[t]$ where $u[t]=K\hat{x}[t]$ is computed from the neural-network percepted states $\hat{x}[t] \approx x[t]$.


## Multiplicative noise model



We model noise by $\hat{x}[t] = Ex[t]$ where $E$ is a diagonal matrix that is close to the identity matrix. Specifically, given a small $\epsilon_i$ such that $\hat{x}_i[t] = (1+\epsilon_i)x_i[t]$,
<!-- given $\lambda_i \approx 1$ such that $\hat{x}_i[t] \in (\frac{x_i[t]}{\lambda_i}, x_i[t]\lambda_i)$ -->
$$
E = \begin{bmatrix}
    1+\epsilon_1 & & \huge0 \\
    & \ddots &  \\
    \huge0 & & 1+\epsilon_n
\end{bmatrix},
$$
and the system model becomes
$$
x[t+1] = Ax[t] + BK(Ex[t]) \\
       = (A+BKE)x[t].
$$

We use a special notation $E_i$ to denote when only **one** state, $x_i$, is affected by uncertainty and all other states are assumed to be perfectly sensed.
$$
E_i = \begin{bmatrix}
    I_{i-1} & & \huge0 \\
    & 1+\epsilon_i &  \\
    \huge0 & & I_{n-i}
\end{bmatrix}.
$$

For example, when $A=
\begin{bmatrix}
a_{11} & a_{11} \\
a_{21} & a_{22}
\end{bmatrix}
$, $B=\begin{bmatrix}
    b_1 \\ b_2
\end{bmatrix}$, and $K = \begin{bmatrix}
    k_1 &  k_2
\end{bmatrix}$,
$$
A+BKE_1 =
\begin{bmatrix}
a_{11}+b_1k_1 (1+\epsilon_1) & a_{11}+b_1k_2 \\
a_{21}+b_2k_1 (1+\epsilon_1) & a_{22}+b_2k_2
\end{bmatrix},
$$
and
$$
A+BKE_2 =
\begin{bmatrix}
a_{11}+b_1k_1 & a_{11}+b_1k_2 (1+\epsilon_2) \\
a_{21}+b_2k_1 & a_{22}+b_2k_2 (1+\epsilon_2)
\end{bmatrix}.
$$

## Computing sensitivity with multiplicative noise model


The key of the technique is to model the system as 
$$x[t+1] = A_\phi x[t]$$
and model the uncertainty in a way such that the system with uncertainty is represented by 
$$x[t+1] = A_\phi' x[t]$$
where $A_\phi' = A_\phi + \epsilon Y + O(\epsilon^2)$. 
Then using $A_\phi$ and $Y$ as inputs, we use the prior technique to compute the sensitivity of maximum singular value of $A_\phi'$ relative to $\epsilon$. Using the previous example, we see that 
$$
\begin{align*}
A_\phi 
&= A+BK \\
&= \begin{bmatrix}
    a_{11}+b_1k_1 & a_{11}+b_1k_2 \\
    a_{21}+b_2k_1 & a_{22}+b_2k_2
\end{bmatrix},
\end{align*}
$$
And for a perturbation in $x_1$, the resulting $A_\phi'$ becomes
$$
\begin{align*}
A_\phi' 
&= A+BKE_1 \\
&=\begin{bmatrix}
    a_{11}+b_1k_1 (1+\epsilon_1) & a_{11}+b_1k_2 \\
    a_{21}+b_2k_1 (1+\epsilon_1) & a_{22}+b_2k_2
\end{bmatrix} \\
&= \begin{bmatrix}
    a_{11}+b_1k_1 & a_{11}+b_1k_2 \\
    a_{21}+b_2k_1 & a_{22}+b_2k_2
\end{bmatrix} +
\epsilon_1 \begin{bmatrix}
    b_1k_1 & 0 \\
    b_2k_1 & 0
\end{bmatrix} \\
&= A_\phi + \epsilon_1 Y_1
\end{align*}
$$
where $Y_1 = \begin{bmatrix}
    b_1k_1 & 0 \\
    b_2k_1 & 0
\end{bmatrix}$. Similarly, $Y_2 =
\begin{bmatrix}
    0 & b_1k_2 \\
    0 & b_2k_2
\end{bmatrix}$.

More generally, for arbitrary $A$, $B$ and $K$, $A_\phi = A+BK$ and
$Y_i = \begin{bmatrix}
    & b_1k_i & \\
    \huge 0 & \vdots & \huge 0 \\
    & b_nk_i & \\
\end{bmatrix}.$

Similarly, for open-loop systems, $A_\phi = A$ and
$Y_i = \begin{bmatrix}
    & a_{1i} & \\
    \huge 0 & \vdots & \huge 0 \\
    & a_{ni} & \\
\end{bmatrix}.$

In [1]:
import numpy as np

def mult_sensitivity(A, B, K, open_loop=False):
    if not open_loop:
        A_phi = A + np.matmul(B, K)
        Y = np.matmul(B, K)
    else:
        A_phi = A
        Y = A

    return A_phi, Y

In [3]:
import control as ctrl
from Benchmarks import sys_variables
from OrderUncertainties import *

def get_sensitivity(A_phi, Y):
    mat = OrdUnc(A_phi)
    if mat.determineCase() == 1:
        return mat.distinctPos(Y)
    else:
        return mat.multSig(Y)

def all_sensitivity(A_phi, Y):
    sensitivity = []
    for i in range(Y.shape[1]):
        Y_i = np.zeros(Y.shape)
        Y_i[:, i] = Y[:, i]
        sensitivity.append(get_sensitivity(A_phi, Y_i))

    # With padded state equations, we emit the last entry in sensitivity.
    return sensitivity

round2g = lambda x: float(f"{x:.2g}")

res = {}
for (name, bench) in sys_variables.items():
    K = ctrl.lqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
    A_phi, Y = mult_sensitivity(bench.sysd.A, bench.sysd.B, K)
    res[name] = list(map(round2g, all_sensitivity(A_phi, Y)))
res

{'RC': [0.03, 0.0029],
 'F1': [0.13, 0.35],
 'DC': [1.3e-05, 0.016],
 'CS': [0.54, 0.075, 0.52, 0.066],
 'EW': [320.0, 0.039],
 'C1': [2e-05],
 'CC': [0.0025, 0.008, 0.049],
 'D5': [0.0, 0.0, 0.0, 0.0, 0.0]}

In [4]:
bench = sys_variables['D5']
A_phi, Y = mult_sensitivity(bench.sysd.A, bench.sysd.B, [], open_loop=True)
list(map(round2g, all_sensitivity(A_phi, Y)))

[0.45, 0.29, 0.48, 0.68, 0.27]